# Análise com SQL Avançado
## U.S. EPA Food Commodity Intake Database (FCID)
### [https://fcid.foodrisk.org/](https://fcid.foodrisk.org/)

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db

# Importando Tabelas do FCID

In [2]:
DROP TABLE IF EXISTS Crop_Group;
DROP TABLE IF EXISTS FCID_Description;
DROP TABLE IF EXISTS Recipes;
DROP TABLE IF EXISTS Intake;

CREATE TABLE Crop_Group (
  CGN VARCHAR(2),
  CGL VARCHAR(6),
  Crop_Group_Description VARCHAR(80),
  PRIMARY KEY (CGL)
) AS SELECT
  CGN, CGL, Crop_Group_Description
FROM CSVREAD('../../data/food-intake/basics/FCID_Cropgroup_Description.csv');

CREATE TABLE FCID_Description (
  CGN VARCHAR(2),
  CG_Subgroup VARCHAR(6),
  FCID_Code VARCHAR(10),
  FCID_Desc VARCHAR(55),
  PRIMARY KEY (FCID_Code),
) AS SELECT
  cgn, CG_Subgroup, FCID_Code, FCID_Desc
FROM CSVREAD('../../data/food-intake/basics/FCID_Code_Description.csv');

CREATE TABLE Recipes (
  Food_Code VARCHAR(8),
  Mod_Code VARCHAR(8),
  Ingredient_Num TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Commodity_Weight DECIMAL(5, 2),
  CSFII_9498_IND TINYINT,
  WWEIA_9904_IND TINYINT,
  WWEIA_0510_IND TINYINT,
  PRIMARY KEY(Food_Code, Mod_Code, Ingredient_Num),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  Food_Code, Mod_Code, Ingredient_Num, FCID_Code, Cooked_Status, Food_Form, Cooking_Method,
  Commodity_Weight, CSFII_9498_IND, WWEIA_9904_IND, WWEIA_0510_IND
FROM CSVREAD('../../data/food-intake/recipes/Recipes_WWEIA_FCID_0510.csv');

CREATE TABLE Intake (
  SeqN INTEGER NOT NULL,
  DayCode TINYINT NOT NULL,
  DraBF TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Intake DECIMAL(13,7),
  Intake_BW DECIMAL(13,10),
  PRIMARY KEY(SeqN, DayCode, FCID_Code, Cooked_Status, Food_Form, Cooking_Method),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  SEQN, DAYCODE, DRABF, FCID_Code, Cooked_Status, Food_Form, Cooking_Method, Intake,Intake_BW
FROM CSVREAD('../../data/food-intake/consumption/Commodity_CSFFM_Intake_0510-cropped.csv');

# Visualizando as Tabelas

In [3]:
SELECT * FROM Crop_Group LIMIT 10;

In [4]:
SELECT * FROM FCID_Description LIMIT 10;

In [5]:
SELECT * FROM Recipes LIMIT 10;

In [6]:
SELECT * FROM Intake LIMIT 10;

# Métricas

Considere que a tabela Intake registra alimentos consumidos por 1.489 pessoas. Considere as seguintes métricas para um alimento:

| Métrica | Descrição |
| --- | --- |
| Popularidade | número de pessoas (dentre as 1.489) que consumiram o alimento |
| Intake_Sum | total consumido do alimento pelas 1.489 pessoas em gramas |
| Intake_AVG | média de consumo do alimento em gramas |
| Intake_AVG_BW | média de consumo do alimento x peso da pessoa |
| Recipes | número de receitas (dentre as 7.154 receitas) que têm o produto como ingrediente |

## 1)  Construa uma View que apresente essas métricas por produto

* Veja exemplo em: `/data/food-intake/computed/commodity-profile.csv`
* Importante: esta tabela foi feita com um número maior de registros, portanto os valores não serão iguais aos seus

In [7]:
DROP VIEW IF EXISTS MetricasFinais;
DROP VIEW IF EXISTS MetricasAlimentos;
DROP VIEW IF EXISTS ReceitasIngredientes;

-- Metricas relacionadas ao consumo e às quantidades
CREATE VIEW MetricasAlimentos AS
    SELECT FCID_CODE, COUNT(*) POPULARITY, SUM(Consumption) INTAKE_SUM,
           SUM(Consumption) / SUM(TimesConsumed) INTAKE_AVG,
           SUM(ConsumptionBW) / SUM(TimesConsumed) INTAKE_BW_AVG
    FROM
        (SELECT FCID_CODE, SeqN, SUM(Intake) Consumption, SUM(Intake_BW) ConsumptionBW, COUNT(*) TimesConsumed
         FROM Intake
         GROUP BY FCID_CODE, SeqN) Alimento
    GROUP BY FCID_CODE;

-- Quantidade de vezes que cada ingrediente aparece em cada receita
CREATE VIEW ReceitasIngredientes AS
    SELECT FCID_Code, COUNT(*) RECIPES
    FROM
        (SELECT FCID_Code
        FROM Recipes
        GROUP BY FCID_Code, Food_Code, Mod_Code) Receitas
    GROUP BY FCID_Code;

-- VIEW final, incluindo dados sobre alimentos e métricas (resposta da questão)
CREATE VIEW MetricasFinais AS
    SELECT F.FCID_CODE, F.FCID_DESC, F.CGN, F.CG_SUBGROUP,
           MA.POPULARITY, MA.INTAKE_SUM, MA.INTAKE_AVG, MA.INTAKE_BW_AVG,
           RI.RECIPES
    FROM FCID_Description F, MetricasAlimentos MA, ReceitasIngredientes RI
    WHERE F.FCID_Code = MA.FCID_Code AND F.FCID_Code = RI.FCID_Code;

In [8]:
-- Metricas relacionadas ao consumo e às quantidades
SELECT * FROM MetricasAlimentos ORDER BY POPULARITY DESC LIMIT 10;

In [9]:
-- Quantidade de vezes que cada ingrediente aparece em cada receita
SELECT * FROM ReceitasIngredientes ORDER BY RECIPES DESC LIMIT 10;

In [10]:
-- VIEW final, incluindo dados sobre alimentos e métricas (resposta da questão)
SELECT * FROM MetricasFinais ORDER BY POPULARITY DESC, FCID_CODE LIMIT 10;

## 2) Como você analisaria a correlação entre as métricas?

* Por exemplo, produtos mais populares são mais consumidos (em número de pessoas ou em quantidade)?
* Proponha uma ou mais queries para fazer esta análise

In [11]:
-- Uma análise que pensamos entre as métricas é "Os alimentos mais populares são os que mais aparecem em receitas?"
-- Para respondermos a essa pergunta, pensamos em normalizar os dados de popularidade e de número de receitas para cada alimento
-- Para fazer essa normalização, basta dividir o valor máximo pelo valor do alimento e multiplicar por 100. O resultado
-- é a porcentagem do valor do alimento em relação ao valor máximo

-- Criando uma view nova para representar os dados
DROP VIEW IF EXISTS PopularidadeReceitas;

CREATE VIEW PopularidadeReceitas AS 
    SELECT FCID_CODE, FCID_DESC, 
           100.0 * POPULARITY / (SELECT MAX(POPULARITY) FROM MetricasFinais) POPULARITY, 
           100.0 * RECIPES / (SELECT MAX(RECIPES) FROM MetricasFinais) RECIPES
    FROM MetricasFinais;

In [12]:
-- Visualizando os dados
SELECT * FROM PopularidadeReceitas LIMIT 10;

In [13]:
-- Com esses dados, é possível analisar os alimentos com maior diferença entre popularidade e receitas
SELECT *, ABS(POPULARITY - RECIPES) DIFFERENCE
    FROM PopularidadeReceitas
    GROUP BY FCID_CODE, FCID_DESC
    ORDER BY DIFFERENCE DESC
    LIMIT 10;

In [14]:
-- E também os com menor diferença - basta alterar o critério de ordenação
SELECT *, ABS(POPULARITY - RECIPES) DIFFERENCE
    FROM PopularidadeReceitas
    GROUP BY FCID_CODE, FCID_DESC
    ORDER BY DIFFERENCE
    LIMIT 10;

## 3) Podemos criar grupos de consumidores conforme um perfil?
* por exemplo, consumidores podem ser agrupados por alimentos que comem predominantemente?
* como você associaria grupos a classes?

In [15]:
-- Podemos agrupar os consumidores com base na variedade de alimentos consumidos.
-- Para cada consumidor diferente, podemos determinar quantos alimentos diferentes foram consumidos por ele.
-- Segundo a nutricionista Steph Geddes, no blog https://www.bodygoodfood.com.au/full-blog/2018/6/19/how-many-foods-should-we-eat-in-a-day
-- uma dieta saudável deve conter de 20 a 30 alimentos diferentes por dia, em média.
-- Assim, como os dados correspondem a dois dias diferentes, podemos separar os consumidores em 3 classes distintas:
--   Classe 1: até 40 alimentos diferentes consumidos
--   Classe 2: de 40 até 60 alimentos diferentes consumidos
--   Classe 3: 60 ou mais alimentos diferentes consumidos

-- Criando uma VIEW com os alimentos distintos consumidos
DROP VIEW IF EXISTS PerfilConsumo;

CREATE VIEW PerfilConsumo AS
    SELECT SEQN, COUNT(DISTINCT FCID_CODE) VARIEDADE
        FROM Intake
        GROUP BY SEQN;

In [16]:
-- Pessoas que pertencem à classe 1 (abaixo do recomendado):
SELECT * FROM PerfilConsumo WHERE VARIEDADE < 40 LIMIT 10;

-- Tamanho da classe: 395
-- SELECT COUNT(*) FROM PerfilConsumo WHERE VARIEDADE < 40;

In [17]:
-- Pessoas que pertencem à classe 2 (dentro do recomendado):
SELECT * FROM PerfilConsumo WHERE VARIEDADE >= 40 AND VARIEDADE <= 60 LIMIT 10;

-- Tamanho da classe: 689
-- SELECT COUNT(*) FROM PerfilConsumo WHERE VARIEDADE >= 40 AND VARIEDADE <= 60;

In [18]:
-- Pessoas que pertencem à classe 3 (acima do recomendado):
SELECT * FROM PerfilConsumo WHERE VARIEDADE > 60 LIMIT 10;

-- Tamanho da classe: 405
-- SELECT COUNT(*) FROM PerfilConsumo WHERE VARIEDADE > 60;

## 4) Que métricas podem ser analisadas para a comparação de perfis?
* escreva uma query SQL que calcule pelo menos uma métrica comparativa

In [19]:
-- A partir do perfil da questão anterior, uma métrica possível que pode ser analisada é a média, em gramas, do consumido por uma pessoa
-- Com esses dados, é possível analisar se há uma correlação, ou não, da quantidade consumida e da variedade consumida

-- Criando uma VIEW para representar a métrica
DROP VIEW IF EXISTS PerfilConsumoComQuantidade;

CREATE VIEW PerfilConsumoComQuantidade AS
    SELECT PC.SEQN, PC.VARIEDADE, SUM(I.Intake) QUANTIDADE
    FROM PerfilConsumo PC, Intake I
    WHERE PC.SEQN = I.SEQN
    GROUP BY PC.SEQN;

In [20]:
-- Visualizando alguns dados
SELECT * FROM PerfilConsumoComQuantidade LIMIT 10;

In [21]:
-- Uma primeira análise possível é em relação aos grupos
-- Para a classe 1 (abaixo do recomendado), a média de quantidade consumida é de 1785.67g
SELECT SUM(QUANTIDADE) / (SELECT COUNT(*) FROM PerfilConsumoComQuantidade WHERE VARIEDADE < 40)
    FROM PerfilConsumoComQuantidade
    WHERE VARIEDADE < 40;
    
-- Para a classe 2 (dentro do recomendado), a média de quantidade consumida é de 2230.88g
SELECT SUM(QUANTIDADE) / (SELECT COUNT(*) FROM PerfilConsumoComQuantidade WHERE VARIEDADE >= 40 AND VARIEDADE <= 60)
    FROM PerfilConsumoComQuantidade
    WHERE VARIEDADE >= 40 AND VARIEDADE <= 60;
    
-- Para a classe 3 (acima do recomendado), a média de quantidade consumida é de 2424.72g
SELECT SUM(QUANTIDADE) / (SELECT COUNT(*) FROM PerfilConsumoComQuantidade WHERE VARIEDADE > 60)
    FROM PerfilConsumoComQuantidade
    WHERE VARIEDADE > 60;
    
-- A partir desses dados, temos que a variedade de alimentos consumidos e a quantidade consumida está, em geral, relacionada
-- O grupo com maior variedade é também o grupo com maior quantidade média consumida

In [22]:
-- Outra análise possível é similar à questão 2. Podemos normalizar a variedade e a quantidade e analisar.
DROP VIEW IF EXISTS VariedadeQuantidade;

CREATE VIEW VariedadeQuantidade AS 
    SELECT SEQN,
           100.0 * VARIEDADE / (SELECT MAX(VARIEDADE) FROM PerfilConsumoComQuantidade) VARIEDADE, 
           100.0 * QUANTIDADE / (SELECT MAX(QUANTIDADE) FROM PerfilConsumoComQuantidade) QUANTIDADE
    FROM PerfilConsumoComQuantidade;

In [23]:
-- Visualizando os dados
SELECT * FROM VariedadeQuantidade LIMIT 10;

In [24]:
-- Pessoas com maior disparidade entre quantidade consumida e variedade
SELECT *, ABS(VARIEDADE - QUANTIDADE) DIFFERENCE
    FROM VariedadeQuantidade
    GROUP BY SEQN
    ORDER BY DIFFERENCE DESC
    LIMIT 10;

In [25]:
-- Pessoas com menor disparidade entre quantidade consumida e variedade
SELECT *, ABS(VARIEDADE - QUANTIDADE) DIFFERENCE
    FROM VariedadeQuantidade
    GROUP BY SEQN
    ORDER BY DIFFERENCE
    LIMIT 10;